In [21]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import wandb
from transformers import LayoutLMv3Processor, LayoutLMv3ForSequenceClassification
from torch.optim import AdamW
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import numpy as np


os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [1]:
import os
from collections import defaultdict

def analyze_dataset_structure(root_dir):
    """Analyze the dataset structure and vendor distribution"""
    vendor_counts = defaultdict(lambda: {'train': 0, 'validation': 0})
    
    for split in ['train', 'validation']:
        split_dir = os.path.join(root_dir, split)
        if not os.path.exists(split_dir):
            continue
            
        for filename in os.listdir(split_dir):
            if not filename.endswith('.png'):
                continue
                
            # Extract vendor name from filename pattern
            vendor_id = filename.split('_')[0]
            vendor_name = '_'.join(filename.split('_')[1:-2])
            vendor_counts[vendor_name][split] += 1
    
    return vendor_counts

def print_dataset_stats(vendor_counts):
    """Print formatted dataset statistics"""
    print("\nDataset Statistics:")
    print("-" * 50)
    print(f"{'Vendor':<30} {'Train':>8} {'Validation':>12} {'Total':>8}")
    print("-" * 50)
    
    total_train = 0
    total_val = 0
    
    for vendor, counts in sorted(vendor_counts.items()):
        train_count = counts['train']
        val_count = counts['validation']
        total = train_count + val_count
        total_train += train_count
        total_val += val_count
        print(f"{vendor:<30} {train_count:>8} {val_count:>12} {total:>8}")
    
    print("-" * 50)
    print(f"{'TOTAL':<30} {total_train:>8} {total_val:>12} {total_train + total_val:>8}")

# Analyze the dataset
dataset_path = "invoice_dataset_processed"
vendor_stats = analyze_dataset_structure(dataset_path)
print_dataset_stats(vendor_stats)


Dataset Statistics:
--------------------------------------------------
Vendor                            Train   Validation    Total
--------------------------------------------------
Brother                              14            3       17
Coople                               22           10       32
Eidg._STVA                            2            0        2
KSU_A-Technik                        29            9       38
K_Müller                             20            5       25
Saviva_AG                            36           11       47
Schaefer_AG                          25            9       34
Shiva_Siegen                         28            9       37
Topmech                              20            6       26
Wei_Grueber                          70           24       94
asa                                  91           36      127
--------------------------------------------------
TOTAL                               357          122      479


In [5]:
import os
import json
from collections import defaultdict

def setup_vendor_mapping():
    """Create and save vendor ID mappings"""
    # Vendor ID mapping (including all vendors)
    vendor_map = {
        "100015806": "Wei_Grueber",
        "15014330": "Shiva_Siegen",
        "15031152": "Topmech",
        "50001213": "KSU_A-Technik",
        "50004790": "Saviva_AG",
        "50005079": "Schaefer_AG",
        "50005828": "Brother",
        "50007702": "K_Müller",
        "50008694": "Eidg._STVA",  # Added this
        "50010889": "asa",
        "50076821": "Coople"
    }
    
    # Create label mappings (excluding Eidg._STVA)
    vendors_to_include = [v for k, v in vendor_map.items() if v != "Eidg._STVA"]
    vendors = sorted(set(vendors_to_include))
    label2id = {vendor: idx for idx, vendor in enumerate(vendors)}
    id2label = {idx: vendor for vendor, idx in label2id.items()}
    
    print("\nVendor to ID mapping:")
    for vendor, idx in label2id.items():
        print(f"{vendor}: {idx}")
        
    return vendor_map, label2id, id2label

def calculate_class_weights(dataset_path, vendor_map, label2id):
    """Calculate class weights to handle imbalance"""
    class_counts = defaultdict(int)
    total_samples = 0
    
    # Count samples per class
    for split in ['train']:  # Only use training set for weights
        split_dir = os.path.join(dataset_path, split)
        for filename in os.listdir(split_dir):
            if not filename.endswith('.png'):
                continue
            vendor_id = filename.split('_')[0]
            vendor_name = vendor_map[vendor_id]
            
            # Skip vendors we're not including in the model
            if vendor_name not in label2id:
                continue
                
            class_counts[vendor_name] += 1
            total_samples += 1
    
    # Calculate weights (inverse of frequency)
    weights = {}
    for vendor in label2id.keys():
        count = class_counts[vendor]
        weights[vendor] = total_samples / (len(class_counts) * count)
    
    return weights

# Setup path constants
DATASET_ROOT = "invoice_dataset_processed"

# Usage example:
vendor_map, label2id, id2label = setup_vendor_mapping()
class_weights = calculate_class_weights(DATASET_ROOT, vendor_map, label2id)

# Print class weights
print("\nClass weights:")
for vendor, weight in sorted(class_weights.items()):
    print(f"{vendor}: {weight:.2f}")


Vendor to ID mapping:
Brother: 0
Coople: 1
KSU_A-Technik: 2
K_Müller: 3
Saviva_AG: 4
Schaefer_AG: 5
Shiva_Siegen: 6
Topmech: 7
Wei_Grueber: 8
asa: 9

Class weights:
Brother: 2.54
Coople: 1.61
KSU_A-Technik: 1.22
K_Müller: 1.77
Saviva_AG: 0.99
Schaefer_AG: 1.42
Shiva_Siegen: 1.27
Topmech: 1.77
Wei_Grueber: 0.51
asa: 0.39


In [47]:
import torch
import torch.nn.functional as F

def prepare_class_weights(class_weights, label2id):
    """Convert class weights dictionary to tensor"""
    # Create tensor of weights in label_id order
    weights = torch.zeros(len(label2id))
    for vendor, weight in class_weights.items():
        weights[label2id[vendor]] = weight
    return weights.to(device)  # Move to same device as model

# Modify the training function to use weights
def train_single_epoch(model, dataloader, optimizer, weights, epoch):
    """Train for one epoch with class weights"""
    model.train()
    total_loss = 0
    num_batches = 0
    
    print(f"\nEpoch {epoch+1}")
    print("-" * 10)
    
    for batch in dataloader:
        # Move batch to device
        batch = {k: v.to(model.device) for k, v in batch.items()}
        
        optimizer.zero_grad()
        outputs = model(**batch)
        
        # Apply class weights to loss
        logits = outputs.logits
        labels = batch['labels']
        
        # print("\nTensor properties before reshape:")
        # print(f"Logits shape: {logits.shape}")
        # print(f"Labels shape: {labels.shape}")
        # print(f"Weights shape: {weights.shape}")
        
        # # Reshape tensors maintaining their dimensions
        # logits = logits.reshape(logits.shape)
        # labels = labels.reshape(labels.shape)
        weights = weights.reshape(weights.shape)
        
        # Try loss calculation
        loss = F.cross_entropy(logits, labels, weight=weights)
        
        # Try loss calculation
        loss = F.cross_entropy(logits, labels, weight=weights)
        
        loss.backward()
        optimizer.step()
        
        current_loss = loss.item()
        total_loss += current_loss
        num_batches += 1
        
        print(f"Batch {num_batches}: Loss = {current_loss:.4f}")
    
    avg_loss = total_loss / num_batches
    print(f"Average loss for epoch {epoch+1}: {avg_loss:.4f}")
    
    return avg_loss

In [48]:
import wandb
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import numpy as np

def evaluate_model_with_metrics(model, dataloader, label2id, id2label):
    """Evaluate model with detailed per-class metrics"""
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in dataloader:
            batch = {k: v.to(model.device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(batch['labels'].cpu().numpy())
    
    # Calculate metrics
    precision, recall, f1, support = precision_recall_fscore_support(
        all_labels, all_preds, labels=list(label2id.values())
    )
    
    # Create per-class metrics dictionary
    class_metrics = {}
    for idx, vendor in id2label.items():
        class_metrics[vendor] = {
            'precision': precision[idx],
            'recall': recall[idx],
            'f1': f1[idx],
            'support': support[idx]
        }
    
    # Calculate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    # Overall accuracy
    accuracy = np.mean(np.array(all_preds) == np.array(all_labels))
    
    return {
        'accuracy': accuracy,
        'class_metrics': class_metrics,
        'confusion_matrix': cm,
        'predictions': all_preds,
        'labels': all_labels
    }

def train_model(model, train_dataloader, val_dataloader, optimizer, class_weights, num_epochs, label2id, id2label):
    """Training loop with detailed wandb logging"""
    # Initialize wandb
    wandb.init(
        project="invoice-classification",
        name="layoutlm-v3-multiclass",
        config={
            "architecture": "LayoutLMv3",
            "dataset": {
                "num_classes": len(label2id),
                "classes": list(label2id.keys()),
                "class_weights": class_weights
            },
            "hyperparameters": {
                "learning_rate": optimizer.param_groups[0]['lr'],
                "epochs": num_epochs,
                "batch_size": train_dataloader.batch_size
            }
        }
    )
    
    # Convert class weights to tensor
    weights = prepare_class_weights(class_weights, label2id)
    
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 50)
        
        # Training Phase
        model.train()
        train_loss = train_single_epoch(model, train_dataloader, optimizer, weights, epoch)
        
        # Evaluation Phase
        print("\nEvaluating...")
        train_metrics = evaluate_model_with_metrics(model, train_dataloader, label2id, id2label)
        val_metrics = evaluate_model_with_metrics(model, val_dataloader, label2id, id2label)
        
        # Log metrics to wandb
        wandb_metrics = {
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_accuracy": train_metrics['accuracy'],
            "val_accuracy": val_metrics['accuracy'],
        }
        
        # Add per-class metrics
        for vendor in label2id.keys():
            # Training metrics
            train_vendor_metrics = train_metrics['class_metrics'][vendor]
            wandb_metrics.update({
                f"train_{vendor}_precision": train_vendor_metrics['precision'],
                f"train_{vendor}_recall": train_vendor_metrics['recall'],
                f"train_{vendor}_f1": train_vendor_metrics['f1']
            })
            
            # Validation metrics
            val_vendor_metrics = val_metrics['class_metrics'][vendor]
            wandb_metrics.update({
                f"val_{vendor}_precision": val_vendor_metrics['precision'],
                f"val_{vendor}_recall": val_vendor_metrics['recall'],
                f"val_{vendor}_f1": val_vendor_metrics['f1']
            })
        
        # Log confusion matrices as plots
        wandb.log({
            **wandb_metrics,
            "train_confusion_matrix": wandb.plot.confusion_matrix(
                probs=None,
                y_true=train_metrics['labels'],
                preds=train_metrics['predictions'],
                class_names=list(label2id.keys())
            ),
            "val_confusion_matrix": wandb.plot.confusion_matrix(
                probs=None,
                y_true=val_metrics['labels'],
                preds=val_metrics['predictions'],
                class_names=list(label2id.keys())
            )
        })
        
        # Print epoch summary
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"Training Loss: {train_loss:.4f}")
        print(f"Training Accuracy: {train_metrics['accuracy']:.4f}")
        print(f"Validation Accuracy: {val_metrics['accuracy']:.4f}")
        
        # Print per-class metrics
        print("\nPer-class Validation Metrics:")
        for vendor, metrics in val_metrics['class_metrics'].items():
            print(f"\n{vendor}:")
            print(f"  Precision: {metrics['precision']:.4f}")
            print(f"  Recall: {metrics['recall']:.4f}")
            print(f"  F1: {metrics['f1']:.4f}")
            print(f"  Support: {metrics['support']}")

    wandb.finish()

In [49]:
from torch.utils.data import Dataset, DataLoader

class InvoiceDataset(Dataset):
    """Dataset for multi-class invoice classification"""
    def __init__(self, samples, processor, vendor_map, label2id):
        self.samples = samples
        self.processor = processor
        self.vendor_map = vendor_map
        self.label2id = label2id
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        # Get sample
        image_path = self.samples[idx]
        vendor_id = os.path.basename(image_path).split('_')[0]
        vendor_name = self.vendor_map[vendor_id]
        label = self.label2id[vendor_name]
        
        # Load and process image
        image = Image.open(image_path).convert("RGB")
        encoding = self.processor(
            image,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512  # Adjust if needed
        )
        
        # Remove batch dimension added by processor
        encoding = {k: v.squeeze(0) for k, v in encoding.items()}
        
        # Add label
        encoding['labels'] = torch.tensor(label)
        
        return encoding

def create_dataloaders(dataset_root, vendor_map, label2id, processor, batch_size=8):
    """Create train and validation dataloaders"""
    train_samples = []
    val_samples = []
    
    # Collect samples
    for split in ['train', 'validation']:
        split_dir = os.path.join(dataset_root, split)
        for filename in os.listdir(split_dir):
            if not filename.endswith('.png'):
                continue
            
            # Skip vendors not in label mapping
            vendor_id = filename.split('_')[0]
            vendor_name = vendor_map[vendor_id]
            if vendor_name not in label2id:
                continue
                
            full_path = os.path.join(split_dir, filename)
            if split == 'train':
                train_samples.append(full_path)
            else:
                val_samples.append(full_path)
    
    # Create datasets
    train_dataset = InvoiceDataset(train_samples, processor, vendor_map, label2id)
    val_dataset = InvoiceDataset(val_samples, processor, vendor_map, label2id)
    
    # Create dataloaders
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True
    )
    
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False
    )
    
    return train_dataloader, val_dataloader

In [ ]:
from transformers import LayoutLMv3ForSequenceClassification, LayoutLMv3Processor
from torch.optim import AdamW

def setup_training(num_labels, label2id, id2label, learning_rate=2e-5):
    """Setup model, processor, and optimizer"""
    # Initialize processor
    processor = LayoutLMv3Processor.from_pretrained("microsoft/layoutlmv3-base")
    
    # Initialize model
    model = LayoutLMv3ForSequenceClassification.from_pretrained(
        "microsoft/layoutlmv3-base",
        num_labels=num_labels,
        label2id=label2id,
        id2label=id2label
    )
    
    # Move model to device (cuda/mps/cpu)
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using MPS device")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using CUDA device")
    else:
        device = torch.device("cpu")
        print("Using CPU")

    # device = torch.device("cpu")
    # print("Using CPU")
    
    model = model.to(device)
    
    # Initialize optimizer
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    
    return model, processor, optimizer, device

# Run setup
print("Setting up training components...")
model, processor, optimizer, device = setup_training(
    num_labels=len(label2id),
    label2id=label2id,
    id2label=id2label
)

Setting up training components...


Some weights of LayoutLMv3ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using CPU


In [51]:
print("Creating dataloaders...")
train_dataloader, val_dataloader = create_dataloaders(
    dataset_root=DATASET_ROOT,
    vendor_map=vendor_map,
    label2id=label2id,
    processor=processor,
    batch_size=8  # Adjust based on your GPU/memory
)

Creating dataloaders...


In [52]:
print("Starting training...")
train_model(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    optimizer=optimizer,
    class_weights=class_weights,
    num_epochs=5,  # Adjust as needed
    label2id=label2id,
    id2label=id2label
)

Starting training...



Epoch 1/5
--------------------------------------------------

Epoch 1
----------


/Users/vaclavpechtor/Code/home_projects/document-processing-demo/env-layoutlm/lib/python3.11/site-packages/transformers/modeling_utils.py:1161: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Batch 1: Loss = 2.4259


/Users/vaclavpechtor/Code/home_projects/document-processing-demo/env-layoutlm/lib/python3.11/site-packages/transformers/modeling_utils.py:1161: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


...

/Users/vaclavpechtor/Code/home_projects/document-processing-demo/env-layoutlm/lib/python3.11/site-packages/transformers/modeling_utils.py:1161: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Batch 45: Loss = 0.0659
Average loss for epoch 5: 0.0901

Evaluating...


/Users/vaclavpechtor/Code/home_projects/document-processing-demo/env-layoutlm/lib/python3.11/site-packages/transformers/modeling_utils.py:1161: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/Users/vaclavpechtor/Code/home_projects/document-processing-demo/env-layoutlm/lib/python3.11/site-packages/transformers/modeling_utils.py:1161: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/Users/vaclavpechtor/Code/home_projects/document-processing-demo/env-layoutlm/lib/python3.11/site-packages/transformers/modeling_utils.py:1161: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/Users/vaclavpechtor/Code/home_projects/document-processing-demo/env-layoutlm/lib/python3.11/site-packages/transformers/modeling_utils.py:1161: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformer


Epoch 5 Summary:
Training Loss: 0.0901
Training Accuracy: 1.0000
Validation Accuracy: 1.0000

Per-class Validation Metrics:

Brother:
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
  Support: 3

Coople:
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
  Support: 10

KSU_A-Technik:
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
  Support: 9

K_Müller:
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
  Support: 5

Saviva_AG:
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
  Support: 11

Schaefer_AG:
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
  Support: 9

Shiva_Siegen:
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
  Support: 9

Topmech:
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
  Support: 6

Wei_Grueber:
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
  Support: 24

asa:
  Precision: 1.0000
  Recall: 1.0000
  F1: 1.0000
  Support: 36


epoch,▁▃▅▆█
train_Brother_f1,▁▇███
train_Brother_precision,▁▇███
train_Brother_recall,▁▁▁▁▁
train_Coople_f1,▁████
train_Coople_precision,▁████
train_Coople_recall,▁▁▁▁▁
train_KSU_A-Technik_f1,▁████
train_KSU_A-Technik_precision,▁▁▁▁▁
train_KSU_A-Technik_recall,▁████
train_K_Müller_f1,▁▁███
